In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import sys
import re
import numpy as np
import openai

from utils import *

os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"

GPT_MODEL = "gpt-4-turbo"  # [gpt-4-turbo]不需要再用preview了，直接上turbo

setOpenAi(keyid = 2)

0

### Least2most
- 其实可以在例子里加上最后一步的subquestion

#### Task: GSM8K

In [3]:
GPT_MODEL = "gpt-4-turbo-preview"  # [gpt-4-turbo-preview]]

problem = """Four years ago, Kody was only half as old as Mohamed. If Mohamed is currently twice 30 years old, how old is Kody?"""

four_shot_Q = [{"role": "user", 
                    "content": """Betty & Paige are raising money for their kids' little league team by hosting a bake sale. Betty has baked 4 dozen chocolate chip cookies, 6 dozen oatmeal raisin cookies and 2 dozen regular brownies. Paige baked 6 dozen sugar cookies, 3 dozen blondies and 5 dozen cream cheese swirled brownies. If they sell the cookies for $1.00 apiece and the blondies/brownies at $2.00 apiece, how much money will they raise?"""}, 
            {"role": "assistant", 
                    "content": """To answer the question "How much money will they raise?", we can decompose it into the following subquestions:
"1. How many dozen cookies did they bake (not including blondies/brownies)?"
"2. How many cookies did they bake (not including blondies/brownies)?"
"3. How many dozen blondies/brownies did they bake (not including cookies)?"
"4. How many blondies/brownies did they bake (not including cookies)?"
"5. How much money will they raise from the cookies (not including blondies/brownies)?"
"6. How much money will they raise from the blondies/brownies (not including cookies)?"
"""},
            
            {"role": "user", 
                    "content": """If Pam is currently twice as young as Rena is, and in 10 years Rena will be 5 years older than her, how old is Pam now?"""}, 
            {"role": "assistant", 
                    "content": """To answer the question "How old is Pam now?", we can decompose it into the following subquestions:
"1. How much older is Rena than Pam currently?"
"""},
            
            {"role": "user", 
                    "content": """As a freelancer, Baylor is paid for every finished work of a client he does on a freelance marketplace. Currently, he has $4000 on his dashboard from previous work done. He is currently working for three clients, with the first client paying him half the amount of money he currently has on his dashboard once the job is done. The second client will pay him 2/5 times more money than the first client once Baylor finishes his work. The third client will pay him twice the amount of money the first and second clients pay him together once he finishes the job. How much money will Baylor have in his dashboard after all the clients pay him for his work?"""}, 
            {"role": "assistant", 
                    "content": """To answer the question "How much money will Baylor have in his dashboard after all the clients pay him for his work?", we can decompose it into the following subquestions:
"1. How much will Baylor's first client pay him for his work?"
"2. How much more will Baylor's second client pay him for his work compared to the first client?"
"3. How much will Baylor's second client pay him for his work?"
"4. How much will the first and second clients pay him together once he finishes the job?"
"5. How much will Baylor's third client pay him for his work?"
"6. How much money will all the clients pay Baylor for his work?"
"""},
            
            {"role": "user", 
                    "content": """Cappuccinos cost $2, iced teas cost $3, cafe lattes cost $1.5 and espressos cost $1 each. Sandy orders some drinks for herself and some friends. She orders three cappuccinos, two iced teas, two cafe lattes, and two espressos. How much change does she receive back for a twenty-dollar bill?"""}, 
            {"role": "assistant", 
                    "content": """To answer the question "How much change does she receive back for a twenty-dollar bill?", we can decompose it into the following subquestions:
"1. How much did the cappuccinos cost in total?"
"2. How much did the iced teas cost in total?"
"3. How much did the cafe lattes cost in total?"
"4. How much did the espressos cost in total?"
"5. How much did the drinks cost in total?"
"""},
            
            {"role": "user", 
                    "content": problem},
            {"role": "assistant", 
                    "content": """Please break down this question into subquestions like the formers. Don't solve. Please begin your answer directly with "1. """}]

decomposed_task = askChatGPT(four_shot_Q, model=GPT_MODEL, temperature = 1)
print(decomposed_task)   


"1. How old is Mohamed currently?"
"2. How old was Kody four years ago?"
"3. How old is Kody currently?"


In [4]:
# 接下来就是把分解完的问题进行询问,以推理得到最终的解答

# 按照换行来划分任务
subtasks = decomposed_task.split('\n')
# 去除每个字符串两端的引号
subtasks = [s.strip('"') for s in subtasks]
# print(subtasks)

systemInfo = {"role": "system", "content": """Here is a question to solve. Subquestions from decomposition are provided. Please follow the subquesitions step by step until reaching the final answer.
The initial question is as follows: {}""".format(problem)}
Q_now = [systemInfo]

for i in range(len(subtasks)):
    subtask = subtasks[i]
    Q_sub = {"role": "user", "content": f"""Subquestion {subtask}"""}
    # print(Q_sub)
    Q_now.append(Q_sub)
    answer_sub = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
    # print(answer_sub)
    A_sub = {"role": "assistant", "content": answer_sub}
    Q_now.append(A_sub)
    # print(Q_now)


Q_final = {"role": "user", "content": """So the final answer is: """}  # 不论subquesition有没有覆盖到最后一问,都统一问一下
Q_now.append(Q_final)
printSeq(Q_now)
answer_final = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
print(answer_final)
    

{'role': 'system', 'content': 'Here is a question to solve. Subquestions from decomposition are provided. Please follow the subquesitions step by step until reaching the final answer.\nThe initial question is as follows: Four years ago, Kody was only half as old as Mohamed. If Mohamed is currently twice 30 years old, how old is Kody?'}
{'role': 'user', 'content': 'Subquestion 1. How old is Mohamed currently?'}
{'role': 'assistant', 'content': "To find Mohamed's current age, given that he is twice 30 years old, we simply multiply 30 by 2. \n\nMohamed's current age is \\(30 \\times 2 = 60\\) years old."}
{'role': 'user', 'content': 'Subquestion 2. How old was Kody four years ago?'}
{'role': 'assistant', 'content': "Four years ago, Kody was only half as old as Mohamed. Given from the previous solution that Mohamed is currently 60 years old, we first need to find out how old Mohamed was four years ago and then use that to find Kody's age at that time.\n\nFour years ago, Mohamed's age would

#### Task: SCAN (one kind of decomposition).
- prompt的格式比较复,有三个部分,任务分解,求解,格式转换

In [3]:
GPT_MODEL = "gpt-4-turbo-preview"  # [gpt-4-turbo-preview]]

problem = """ "turn right after run left thrice" """

decom_system = {"role": "system", "content": """I will give you a piece of natural language command. I need you to decompose it to smaller commands.
8 examples are as follows:

Q: "look right after look twice"
A: "look right after look twice" can be solved by: "look right", "look twice".

Q: "jump opposite right thrice and walk"
A: "jump opposite right thrice" can be solved by: "jump opposite right", "jump opposite right thrice". "walk" can be solved by: "walk". So, "jump opposite right thrice and walk" can be solved by: "jump opposite right", "jump opposite right thrice", "walk".

Q: "run left twice and run right"
A: "run left twice" can be solved by: "run left", "run left twice". "run right" can be solved by "run right". So, "run left twice and run right" can.be solved by: "run left", "run left twice", "run right".

Q: "run opposite right"
A: "run opposite right" can be solved by "run opposite right".

Q: "look opposite right thrice after walk"
A: "look opposite right thrice" can be solved by: "look opposite right", "look opposite right thrice". "walk" can be solved by "walk". So, "look opposite right thrice after walk" can be solved by: "look opposite right", "look opposite right thrice", "walk".

Q: "jump around right"
A: "jump around right" can be solved by: "jump right", "jump around right". So, "jump around right" can be solved by: "jump right", "jump around right".

Q: "look around right thrice and walk"
A: "look around right thrice" can be solved by: "look right", "look around right", "look around right thrice". "walk" can be solved by "walk". So, "look around right thrice and walk" can be solved by: "look right", "look around right", "look around right thrice", "walk".

Q: "turn right after run right thrice"
A: "turn right" can be solved by: "turn right". "run right thrice" can be solved by: "run right", "run right thrice". So, "turn right after run right thrice" can be solved by: "turn right", "run right", "run right thrice".
"""}

question = {"role": "user", "content": f"""Now the command is {problem}, please decompose it into smaller commands like the examples.
Format: "Command" can be solved by: "subcommand_0", "subcommand_1",...  """}
decom_Q = [decom_system, question]
decom_commands = askChatGPT(decom_Q, model=GPT_MODEL, temperature = 1)
print(decom_commands)

# 正则表达式匹配 can be solved by: 之后的引号中的内容
pattern = r'can be solved by:\s*("[^"]*"(?:,\s*"[^"]*")*)'

# 查找匹配项
match = re.search(pattern, decom_commands)

# 提取匹配内容并转换为列表
if match:
    # 提取逗号分隔的引号中的内容
    commands_decomposed = re.findall(r'"([^"]*)"', match.group(1))
    print(commands_decomposed)
else:
    print("没有找到匹配的内容")


"turn right after run left thrice" can be solved by: "run left", "run left thrice", "turn right".
['run left', 'run left thrice', 'turn right']


#### 定义图上的元关系
#### 然后再将文字关系映射到抽象关系图

In [4]:
problem
commands_decomposed
connect_system = {"role": "system", "content": """Now we have a natural language instruction, which we have broken down into many subcommands in the form of a list. I want you to understand the connection between these subcommands."""}
connect_user = {"role": "user", "content":f"The init natural language instruction is {problem}. And the subcommands are {commands_decomposed} Please provide your understanding of the relationships between these subcommands."}
connect_Q = [connect_system, connect_user]
connect_commands = askChatGPT(connect_Q, model=GPT_MODEL, temperature = 1)

print(connect_commands)

Given the natural language instruction "turn right after run left thrice" and the provided subcommands ['run left', 'run left thrice', 'turn right'], here's the breakdown of the relationships between these subcommands:

1. **'run left'**: This command represents a single action of moving or running to the left. It serves as the base action that is repeated in the instruction.

2. **'run left thrice'**: This command elaborates on the base action ('run left') by indicating that it should be performed three times in succession. It specifies the frequency or repetition of the base action within the context of the overall instruction. The relationship between 'run left' and 'run left thrice' is that the latter is essentially an extension or multiplication of the former, indicating not just a single execution but three consecutive executions of the action.

3. **'turn right'**: This command is an action that follows the repeated action ('run left thrice'). Its relationship with 'run left thr

In [5]:
# 定义图的元操作: 聚合
element_system = {"role":"system", "content":"""Now we need to create a standardized model for the relationships between subcommands. We have defined two basic relationships:
1. Progressive: This means that subcommand 2 is the direct subsequent step to subcommand 1, providing a detailed explanation and additional requirements for subcommand 1. The result of subcommand 2 can directly override the result of subcommand 1 for subsequent reasoning.
2. Parallel: This means that subcommand 1 and subcommand 2 are two parallel steps. They are independent, with results that are somewhat related but not interchangeable. They can be handled separately and then aggregated.
"""}
element_Q =  {"role": "user", "content":"""Now please rewrite your understanding of the relationship between subcommands using 'progressive' and 'parallel'.
Answer format: 
1. "xxx1" progressive "xxx2"
2. "xxx2" parallel "xxx3"
3. ...

Notice that "xxx1" progressive "xxx2" means "xxx2" is the subsequent step to "xxx1".
"""}
connect_Q.append(element_system)
connect_Q.append(element_Q)
tuple_subcommands = askChatGPT(connect_Q, model=GPT_MODEL, temperature = 1)
print(tuple_subcommands)

1. "run left" progressive "run left thrice"
2. "run left thrice" parallel "turn right"


#### 将文本化的描述转化为结构化的推理图 - 如何建图?

In [7]:
# print(connect_commands)

# 正则表达式模式,从语言描述中提取关系三元组
pattern = r'"\s*([^"]+)\s*"\s*(\w+)\s*"\s*([^"]+)\s*"'
# 使用正则表达式查找所有匹配的三元组
matches = re.findall(pattern, tuple_subcommands)
# 输出提取的关系三元组
for match in matches:
    print(match)


('run left', 'progressive', 'run left thrice')
('run left thrice', 'parallel', 'turn right')


In [5]:
from graphviz import Digraph

# 定义关系描述的三元组
relationships = [
    ('run left', 'progressive', 'run left thrice'),
    ('run left thrice', 'parallel', 'turn right')
]

# 创建有向图
dot = Digraph(comment='Process Flow')

# 用于存储并行关系的集合
parallel_pairs = []

# 添加节点和边
for obj1, relation, obj2 in relationships:
    dot.node(obj1, obj1)
    dot.node(obj2, obj2)
    
    if relation == 'progressive':
        dot.edge(obj1, obj2, label='progressive')
    elif relation == 'parallel':
        parallel_pairs.append((obj1, obj2))

# 添加并行关系的无箭头虚线和共同指向的result节点
if parallel_pairs:
    with dot.subgraph() as s:
        for obj1, obj2 in parallel_pairs:
            s.node(obj1, obj1)
            s.node(obj2, obj2)
            s.edge(obj1, obj2, style='dotted', arrowhead='none', label='parallel')
        s.node('result', 'result')
        for obj1, obj2 in parallel_pairs:
            s.edge(obj1, 'result', style='dotted') # arrowhead='none'
            s.edge(obj2, 'result', style='dotted') # arrowhead='none'

# 保存并展示图表，增加分辨率参数
dot.attr(dpi='300')  # 设置dpi参数提高分辨率
dot.render('process_flow', format='png', view=True)


'process_flow.png'

#### 求解subcommand,将subcommand映射到具体的action序列

In [41]:


map_system= {"role": "system", "content": f"""I need you to map a piece of natural language command to action sequences. I've broken command down into smaller subcommands for you. Please follow the subcommands I provided.
The initial command is {commands_decomposed}.
13 examples are as follows:

Q: "turn left"
A: "turn left" outputs "TURN LEFT".

Q: "turn right"
A: "turn right" outputs "TURN RIGHT".

Q: "jump left"
A: The output of "jump left" concatenates: the output of "turn left", the output of "jump". "turn left" outputs "TURN LEFT". "jump" outputs "JUMP". So concatenating the output of "turn left" and the output of "jump" leads to "TURN LEFT" + "JUMP". So the output of "jump left" is "TURN LEFT" + "JUMP".

Q: "run right"
A: The output of "run right" concatenates: the output of "turn right", the output of "run". "turn right" outputs "TURN RIGHT". "run" outputs "RUN". So concatenating the output of "turn right" and the output of "run" leads to "TURN RIGHT" + "RUN". So the output of "run right" is "TURN RIGHT" + "RUN".

Q: "look twice"
A: The output of "look twice" concatenates: the output of "look", the output of "look". "look" outputs "LOOK". So repeating the output of "look" two times leads to "LOOK" * 2. So the output of "look twice" is "LOOK" * 2.

Q: "run and look twice"
A: The output of "run and look twice" concate+nates: the output of "run", the output of "look twice". "run" outputs "RUN". "look twice" outputs "LOOK" * 2. So concatenating the output of "run" and the output of "look twice" leads to "RUN" + "LOOK" * 2. So the output of "run and look twice" is "RUN" + "LOOK" * 2.

Q: "jump right thrice"
A: The output of "jump right thrice" concatenates: the output of "jump right", the output of "jump right", the output of "jump right". "jump right" outputs "TURN RIGHT" + "JUMP". So repeating the output of "jump right" three times leads to ("TURN RIGHT" + "JUMP") * 3. So the output of "jump right thrice" is ("TURN RIGHT" + "JUMP") * 3.

Q: "walk after run"
A: The output of "walk after run" concatenates: the output of "run", the output of "walk". "run" outputs "RUN". "walk" outputs "WALK". So concatenating the output of "run" and the output of "walk" leads to "RUN" + "WALK". So the output of "walk after run" is "RUN" + "WALK".

Q: "turn opposite left"
A: The output of "turn opposite left" concatenates: the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" twice leads to "TURN LEFT" * 2. So the output of "turn opposite left" is "TURN LEFT" * 2.

Q: "turn around left"
A: The output of "turn around left" concatenates: the output of "turn left", the output of "turn left", the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" four times leads to "TURN LEFT" * 4. So the output of "turn around left" is "TURN LEFT" * 4. Q: "turn opposite right" A: The output of "turn opposite right" concatenates: the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" twice leads to "TURN RIGHT" * 2. So the output of "turn opposite right" is "TURN RIGHT" * 2.

Q: "turn around right"
A: The output of "turn around right" concatenates: the output of "turn right", the output of "turn right", the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" four times leads to "TURN RIGHT" * 4. So the output of "turn around right" is "TURN RIGHT" * 4.

Q: "walk opposite left"
A: The output of "walk opposite left" concatenates: the output of "turn opposite left", the output of "walk". "turn opposite left" outputs "TURN LEFT" * 2. "walk" outputs "WALK". So concatenating the output of "turn opposite left" and the output of "walk" leads to "TURN LEFT" * 2 + "WALK". So the output of "walk opposite left" is "TURN LEFT" * 2 + "WALK".

Q: "walk around left"
A: The output of "walk around left" concatenates: the output of "walk left", the output of "walk left", the output of "walk left", the output of "walk left". "walk left" outputs "TURN LEFT" + "WALK". So repeating the output of "walk around left" four times leads to ("TURN LEFT" + "WALK") * 4. So the output of "walk around left" is ("TURN LEFT" + "WALK") * 4.
"""}

Q_now = [map_system]

for i in range(len(commands_decomposed)):
    subcommand = commands_decomposed[i]
    Q_sub = {"role": "user", "content": f"""Subcommand: {subcommand}. Please map the subcommand to actions."""}
    Q_now.append(Q_sub)
    answer_sub = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
    print(answer_sub)
    Q_now.append({"role": "assistant", "content": answer_sub})

Q_final = {"role": "user", "content": f"""Now you need to summarize the final action sequence based on the results of the previous subcommand. Note that the final result is not necessarily a concatenation of all subcommands results.
Example1: "turn right after run right thrice" can be solved by: "turn right", "run right", "run right thrice". But "run right thrice" is the subsequent step behind "run right" and it is only needed to concatenate the actions of "run right thrice" with the actions of "turn right".

The initial command is {commands_decomposed}. Consider the connections between the various subcommands, please answer the final answer. There is no need to break multiplication into addition."""}  # 不论subquesition有没有覆盖到最后一问,都统一问一下
Q_now.append(Q_final)
# printSeq(Q_now)
answer_final = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
print(answer_final)
Q_now.append({"role": "assistant", "content": answer_final})

Q_simplify = {"role": "user", "content": """Please only output the action sequence and no explanations are needed. Example: "TURN LEFT" * 2 + "WALK" """}
Q_now.append(Q_simplify)
answer_simplify = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
print('\n'+answer_simplify)

The output of "run left" concatenates: the output of "turn left", the output of "run". "turn left" outputs "TURN LEFT". "run" outputs "RUN". So concatenating the output of "turn left" and the output of "run" leads to "TURN LEFT" + "RUN". So the output of "run left" is "TURN LEFT" + "RUN".
The output of "run left thrice" concatenates: the output of "run left", the output of "run left", the output of "run left". "run left" outputs "TURN LEFT" + "RUN". So repeating the output of "run left" three times leads to ("TURN LEFT" + "RUN") * 3. So the output of "run left thrice" is ("TURN LEFT" + "RUN") * 3.
"turn right" outputs "TURN RIGHT".
Given the sequence and understanding the redundancy in repeating actions, we start with "run left", which is then expanded into "run left thrice", and finally, there is the action "turn right". Since "run left thrice" inherently includes the action of "run left" done three times, we only need to perform "run left thrice" followed by "turn right".

So, combin

In [47]:
# 接下来就是要把这个伪式子给转成action的序列


rewrite_system = {"role": "system", "content": f"""Now I have a pseudo action sequence expression with parentheses and multiplication. I need you to help me change this into a sequence of actions without an operator sign.
6 examples are as follows:    
    
Q: “JUMP” * 3
Rewrite: “JUMP” * 3
A: 1 JUMP 2 JUMP 3 JUMP

Q: “RUN” * 4 * 2
Rewrite: “RUN” * 8
A: 1 RUN 2 RUN 3 RUN 4 RUN 5 RUN 6 RUN 7 RUN 8 RUN

Q: “TURN RIGHT” + “WALK”
Rewrite: “TURN RIGHT” + “WALK”
A: TURN RIGHT WALK

Q: (“TURN LEFT” + “LOOK”) * 2 + “TURN LEFT” + “LOOK”
Rewrite: (“TURN LEFT” + “LOOK”) * 2 + “TURN LEFT” + “LOOK”
A: 1 (TURN LEFT LOOK) 2 (TURN LEFT LOOK) TURN LEFT LOOK

Q: (“TURN RIGHT” * 2 + “JUMP”) * 4
Rewrite: (“TURN RIGHT” * 2 + “JUMP”) * 4
A: 1 (1 TURN RIGHT 2 TURN RIGHT JUMP) 2 (1 TURN RIGHT 2 TURN RIGHT JUMP) 3 (1 TURN RIGHT 2 TURN RIGHT JUMP) 4 (1 TURN RIGHT 2 TURN RIGHT JUMP)

Q: “TURN LEFT” * 2 + (“TURN RIGHT” + “WALK”) * 4 * 2
Rewrite: “TURN LEFT” * 2 + (“TURN RIGHT” + “WALK”) * 8
A: 1 TURN LEFT 2 TURN LEFT 1 (TURN RIGHT WALK) 2 (TURN RIGHT WALK) 3 (TURN RIGHT WALK) 4 (TURN RIGHT WALK) 5 (TURN RIGHT WALK) 6 (TURN RIGHT WALK) 7 (TURN RIGHT WALK) 8 (TURN RIGHT WALK)
"""}

Q_change = {"role": "user", "content": f"""The pseudo action sequence expression is {answer_simplify} Please change it to the action sequences."""}
Q_now = [rewrite_system, Q_change]
actions = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
print(actions)

# 删除数字
text_no_numbers_and_brackets = re.sub(r'[\d\(\)]', '', actions)
# 替换 TURN 后面的空格为下划线
text_formatted = re.sub(r'TURN\s+', 'TURN_', text_no_numbers_and_brackets)
# 去掉多余的空格
text_formatted = re.sub(r'\s+', ' ', text_formatted).strip()
print(text_formatted)

1 (TURN LEFT RUN) 2 (TURN LEFT RUN) 3 (TURN LEFT RUN) TURN RIGHT
TURN_LEFT RUN TURN_LEFT RUN TURN_LEFT RUN TURN_RIGHT
